In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (15,15)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df_loc = pd.read_csv("../training_real_dataset_450000.csv", sep = ",")

In [57]:
df_loc["lon.1"] = (df_loc["lon.1"] - min(df_loc["lon.1"]))/(max(df_loc["lon.1"]) - min(df_loc["lon.1"]))
df_loc["lat.1"] = (df_loc["lat.1"] - min(df_loc["lat.1"]))/(max(df_loc["lat.1"]) - min(df_loc["lat.1"]))

In [3]:
df_loc.head(3)

,Unnamed: 0,lon,lat,Z1,Z2,lon.1,lat.1,optional
0,362707,53.031696,26.386187,-4.469473,5.580139,53.031696,26.386187,True
1,204031,41.248749,17.121719,-4.429966,3.125276,41.248749,17.121719,True
2,461073,50.063469,32.142750,-1.551609,-2.033521,50.063469,32.142750,True


In [5]:
N = len(df_loc)
s = np.vstack((df_loc["lon.1"],df_loc["lat.1"])).T
y = np.array(df_loc[["Z1","Z2"]])

In [7]:
start_time = time.time()
num_basis = [10**2,19**2,37**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [8]:
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []
s_train, s_test,x_train,x_test,y_train, y_test= train_test_split(s, phi, y, 
                                                            test_size=0.1)
#     data_train = np.hstack((encoder_train,y_train))
#     n_rows = data_train.shape[0]
#     random_indices = np.random.choice(n_rows, size=10000, replace=True)
#     resampled_data_train = data_train[random_indices, :]
    # DeepKriging model for continuous data
model = Sequential()
# model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(100, input_dim = x_train.shape[1],  
                kernel_initializer='he_uniform', activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dropout(rate=0.5))
#     model.add(Dense(10, activation='relu'))
#model.add(BatchNormalization())
model.add(Dense(2, activation='linear'))
NB_START_EPOCHS = 50 
# NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
BATCH_SIZE = 64  
fold_no = 1
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])

result = model.fit(x_train, y_train,
                   validation_data=(x_test,y_test), epochs = 100, batch_size = 512, verbose = 2)
callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             ModelCheckpoint(filepath='Biv_nonStationary_model.h5', monitor='val_loss', save_best_only=True)]
result = model.fit(x_train, y_train, callbacks=callbacks, 
                   validation_data=(x_test,y_test), epochs = 50, batch_size = 512, verbose = 2)

model = keras.models.load_model('Biv_nonStationary_model.h5')
y_pred = model.predict(x_test)

# Mean Squared Error
mse_var1.append(mean_squared_error(y_test[:,0], y_pred[:,0]))
mse_var2.append(mean_squared_error(y_test[:,1], y_pred[:,1]))
end_time = time.time()
print("%s seconds", end_time - start_time)

Epoch 1/100
792/792 - 33s - loss: 0.6505 - mse: 0.6505 - mae: 0.5050 - val_loss: 0.4764 - val_mse: 0.4764 - val_mae: 0.4280
Epoch 2/100
792/792 - 26s - loss: 0.4422 - mse: 0.4422 - mae: 0.4120 - val_loss: 0.5781 - val_mse: 0.5781 - val_mae: 0.5197
Epoch 3/100
792/792 - 24s - loss: 0.4018 - mse: 0.4018 - mae: 0.3907 - val_loss: 0.3965 - val_mse: 0.3965 - val_mae: 0.3961
Epoch 4/100
792/792 - 31s - loss: 0.3765 - mse: 0.3765 - mae: 0.3784 - val_loss: 0.3787 - val_mse: 0.3787 - val_mae: 0.3909
Epoch 5/100
792/792 - 26s - loss: 0.3532 - mse: 0.3532 - mae: 0.3645 - val_loss: 0.6073 - val_mse: 0.6073 - val_mae: 0.5442
Epoch 6/100
792/792 - 14s - loss: 0.3410 - mse: 0.3410 - mae: 0.3573 - val_loss: 0.4333 - val_mse: 0.4333 - val_mae: 0.4432
Epoch 7/100
792/792 - 11s - loss: 0.3263 - mse: 0.3263 - mae: 0.3497 - val_loss: 0.3240 - val_mse: 0.3240 - val_mae: 0.3506
Epoch 8/100
792/792 - 12s - loss: 0.3164 - mse: 0.3164 - mae: 0.3428 - val_loss: 0.3096 - val_mse: 0.3096 - val_mae: 0.3371
Epoch 9/

Epoch 67/100
792/792 - 20s - loss: 0.1653 - mse: 0.1653 - mae: 0.2497 - val_loss: 0.1787 - val_mse: 0.1787 - val_mae: 0.2539
Epoch 68/100
792/792 - 26s - loss: 0.1686 - mse: 0.1686 - mae: 0.2523 - val_loss: 0.2031 - val_mse: 0.2031 - val_mae: 0.2896
Epoch 69/100
792/792 - 26s - loss: 0.1630 - mse: 0.1630 - mae: 0.2486 - val_loss: 0.2403 - val_mse: 0.2403 - val_mae: 0.3409
Epoch 70/100
792/792 - 24s - loss: 0.1634 - mse: 0.1634 - mae: 0.2502 - val_loss: 0.1787 - val_mse: 0.1787 - val_mae: 0.2609
Epoch 71/100
792/792 - 18s - loss: 0.1594 - mse: 0.1594 - mae: 0.2466 - val_loss: 0.2128 - val_mse: 0.2128 - val_mae: 0.3046
Epoch 72/100
792/792 - 14s - loss: 0.1595 - mse: 0.1595 - mae: 0.2467 - val_loss: 0.2746 - val_mse: 0.2746 - val_mae: 0.3663
Epoch 73/100
792/792 - 23s - loss: 0.1602 - mse: 0.1602 - mae: 0.2473 - val_loss: 0.1893 - val_mse: 0.1893 - val_mae: 0.2801
Epoch 74/100
792/792 - 22s - loss: 0.1596 - mse: 0.1596 - mae: 0.2472 - val_loss: 0.1956 - val_mse: 0.1956 - val_mae: 0.2819


In [22]:
df_loc = pd.read_csv("../testing_real_dataset_450000.csv", sep = ",")

In [12]:
df_loc["lon.1"] = (df_loc["lon.1"] - min(df_loc["lon.1"]))/(max(df_loc["lon.1"]) - min(df_loc["lon.1"]))
df_loc["lat.1"] = (df_loc["lat.1"] - min(df_loc["lat.1"]))/(max(df_loc["lat.1"]) - min(df_loc["lat.1"]))

In [13]:
N = len(df_loc)
s = np.vstack((df_loc["lon.1"],df_loc["lat.1"])).T
y = np.array(df_loc[["Z1","Z2"]])
num_basis = [10**2,19**2,37**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [14]:
y_pred = model.predict(phi)

In [15]:
y_pred.shape

(56771, 2)

In [16]:
mse_var1 = (y_pred[:,0] - df_loc["Z1"])**2
mse_var2 = (y_pred[:,1] - df_loc["Z2"])**2


In [17]:
np.vstack((mse_var1,mse_var2)).T

array([[0.00366084, 0.12044814],
       [0.28052682, 0.27122343],
       [0.0087143 , 0.17702045],
       ...,
       [0.03869813, 1.05857149],
       [1.00854306, 0.58483528],
       [1.1763527 , 0.68286354]])

In [18]:
df_test_loc = pd.DataFrame(s, columns = ["x","y"])
df_val = pd.DataFrame(np.vstack((mse_var1,mse_var2)).T, columns = ["mse1","mse2"])
df = pd.concat([df_test_loc,df_val], axis = 1)
df.to_csv("../real_data_450000_mse.csv")

In [23]:
df_test_loc = pd.DataFrame(np.vstack((df_loc["lon.1"],df_loc["lat.1"])).T, columns = ["x","y"])
df_val = pd.DataFrame(y_pred, columns = ["var1","var2"])
df = pd.concat([df_test_loc,df_val], axis = 1)
df.to_csv("../real_data_450000_predictions.csv")

In [96]:
df_loc1 = pd.read_csv("../interpolation_location.csv", sep = ",")

In [92]:
df_loc1["Var1"] = (df_loc1["Var1"] - min(df_loc["lon.1"]))/(max(df_loc["lon.1"]) - min(df_loc["lon.1"]))
df_loc1["Var2"] = (df_loc1["Var2"] - min(df_loc["lat.1"]))/(max(df_loc["lat.1"]) - min(df_loc["lat.1"]))

In [93]:
N = len(df_loc1)
s = np.vstack((df_loc1["Var1"],df_loc1["Var2"])).T
num_basis = [10**2,19**2,37**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [94]:
y_pred_interpolation = model.predict(phi)


In [97]:
df_test_loc = pd.DataFrame(np.vstack((df_loc1["Var1"],df_loc1["Var2"])).T, columns = ["x","y"])
df_val = pd.DataFrame(y_pred_interpolation, columns = ["var1","var2"])
df = pd.concat([df_test_loc,df_val], axis = 1)
df.to_csv("../../plot_results/real_data_interpolation.csv")

In [98]:
df

,x,y,var1,var2
0,34.727631,27.015812,3.836477,-0.731951
1,34.732003,27.015812,3.788488,-0.752170
2,34.736376,27.015812,3.740386,-0.772466
3,34.740748,27.015812,3.692175,-0.792835
4,34.745121,27.015812,3.643853,-0.813280
...,...,...,...,...
21020,35.339779,28.994629,3.683045,-1.761876
21021,35.344152,28.994629,3.694591,-1.785215
21022,35.348524,28.994629,3.720402,-1.808304
21023,35.352897,28.994629,3.745299,-1.832900
